In [1]:
# !pip install tez

In [24]:
import gc
gc.enable()



import os
from IPython.core.display import display, HTML
import numpy as np
import pandas as pd
import tez
import torch
import torch.nn as nn
from joblib import Parallel, delayed
from transformers import AutoConfig, AutoModel, AutoTokenizer
from torch.utils.data import Dataset,DataLoader
import glob
from timeit import default_timer as timer
from torch.utils.data.sampler import *
import torch.cuda.amp as amp
from torch.nn.parallel.data_parallel import data_parallel

import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"

In [3]:
def time_to_str(t, mode='min'):
    if mode=='min':
        t  = int(t)/60
        hr = t//60
        min = t%60
        return '%2d hr %02d min'%(hr,min)
    elif mode=='sec':
        t   = int(t)
        min = t//60
        sec = t%60
        return '%2d min %02d sec'%(min,sec)
    else:
        raise NotImplementedError

In [4]:
discourse_marker_to_label = {
    "B-Lead": 0,
    "I-Lead": 1,
    "B-Position": 2,
    "I-Position": 3,
    "B-Evidence": 4,
    "I-Evidence": 5,
    "B-Claim": 6,
    "I-Claim": 7,
    "B-Concluding Statement": 8,
    "I-Concluding Statement": 9,
    "B-Counterclaim": 10,
    "I-Counterclaim": 11,
    "B-Rebuttal": 12,
    "I-Rebuttal": 13,
    "O": 14,
    "PAD": -100,
}


label_to_discourse_marker = {v: k for k, v in discourse_marker_to_label.items()}

num_discourse_marker = 15 

probability_threshold = {
    "Lead": 0.687,
    "Position": 0.537,
    "Evidence": 0.637,
    "Claim": 0.537,
    "Concluding Statement": 0.687,
    "Counterclaim": 0.537,
    "Rebuttal": 0.537,
 
}

length_threshold = {
    "Lead": 7,
    "Position": 3,
    "Evidence": 12,
    "Claim": 1,
    "Concluding Statement": 9,
    "Counterclaim": 4,
    "Rebuttal": 2,
}
max_length = 1600
is_amp = True

In [5]:
model_grps = {


    
    'deberta':{
        'arch' : '../input/debertalarge',
        'batch_size':8,
        'max_length' : max_length,
        'checkpoints':
        [
            ('../input/deberta-large-5-fold/model_0.bin','deberta_fold_0'),

        ]    
    },
    
}

In [6]:
def tokenize_data(texts, tokenizer, max_length = 4096):
    results = []

    for id_, text in texts:
        encoded = tokenizer.encode_plus(
            text,
            add_special_tokens=False,
            return_offsets_mapping=True,
            max_length=1600,  # <todo>
            truncation=True,
        )
        token_id = encoded['input_ids']
        token_offset = encoded['offset_mapping']

        # add end, start token id
        token_id = [tokenizer.cls_token_id] + token_id
        token_id = token_id[: max_length - 1]  # need to set as 4096, do not abandon tokens
        token_id = token_id + [tokenizer.sep_token_id]

        # padding
        token_mask = [1] * len(token_id)

        #     padding_length = max_length - len(token_id)
        #     if padding_length > 0:
        #         if tokenizer.padding_side == 'right':
        #             token_id    = token_id    + [tokenizer.pad_token_id] * padding_length
        #             token_mask  = token_mask  + [0] * padding_length
        #         else:
        #             raise NotImplementedError

        results.append((id_, text, token_offset, token_id, token_mask))

    return results

In [7]:
class FeedbackDataset(Dataset):
    def __init__(self, tokenized_data):
        self.tokenized_data = tokenized_data
        self.length = len(self.tokenized_data)

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        # text to token

        id, text, token_offset, token_id, token_mask = self.tokenized_data[index]

        # -------------------------------------
        r = {}
        r['index'] = index
        r['id'] = id
        r['text'] = text
        r['token_offset'] = str(token_offset)  # force batch loader store as list
        #         r['token_id'    ] = torch.tensor(token_id,    dtype=torch.long)
        #         r['token_mask'  ] = torch.tensor(token_mask,  dtype=torch.long)
        r['token_id'] = token_id
        r['token_mask'] = token_mask

        return r


class FeedbackDatasetValid:
    def __init__(self, samples, max_len, tokenizer):
        self.samples = samples
        self.max_len = max_len
        self.tokenizer = tokenizer
        self.length = len(samples)

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        input_ids = self.samples[idx]["input_ids"]
        input_ids = [self.tokenizer.cls_token_id] + input_ids

        if len(input_ids) > self.max_len - 1:
            input_ids = input_ids[: self.max_len - 1]

        # add end token id to the input_ids
        input_ids = input_ids + [self.tokenizer.sep_token_id]
        attention_mask = [1] * len(input_ids)

        return {
            "ids": input_ids,
            "mask": attention_mask,
        }


class Collate:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def __call__(self, batch):
        output = dict()
        output["token_id"] = [sample["token_id"] for sample in batch]
        output["token_mask"] = [sample["token_mask"] for sample in batch]
        output["index"] = [sample['index'] for sample in batch]
        output["id"] = [sample["id"] for sample in batch]
        output["text"] = [sample["text"] for sample in batch]
        output["token_offset"] = [sample["token_offset"] for sample in batch]


        # calculate max token length of this batch
        batch_max = max([len(ids) for ids in output["token_id"]])
        # batch_max = 4096

        # add padding
        if self.tokenizer.padding_side == "right":
            output["token_id"] = [s + (batch_max - len(s)) * [self.tokenizer.pad_token_id] for s in output["token_id"]]
            output["token_mask"] = [s + (batch_max - len(s)) * [0] for s in output["token_mask"]]
        else:
            output["token_id"] = [(batch_max - len(s)) * [self.tokenizer.pad_token_id] + s for s in output["token_id"]]
            output["token_mask"] = [(batch_max - len(s)) * [0] + s for s in output["token_mask"]]

        # convert to tensors
        output["token_id"] = torch.tensor(output["token_id"], dtype=torch.long)
        output["token_mask"] = torch.tensor(output["token_mask"], dtype=torch.long)

        return output

In [8]:
valid_id = [ f.replace("\\", "/").split('/')[-1][:-4] for f in glob.glob('../input/feedback-prize-2021/test'+'/*.txt') ] # get file names
# valid_id = pd.read_csv('../input/feedback-prize-2021/train.csv').id.unique().tolist()
valid_id = sorted(valid_id)
num_valid = len(valid_id)
print('len(valid_id)',len(valid_id))

len(valid_id) 5


In [9]:
df_text=[]
for id in valid_id:
    text_file = '../input/feedback-prize-2021/test' +'/%s.txt'%id
    with open(text_file, 'r') as f:
        text = f.read()

    text = text.replace(u'\xa0', u' ')
    text = text.rstrip()
    text = text.lstrip()
    df_text.append((id,text))
df_text = pd.DataFrame(df_text, columns=['id','text'])
df_text['text_len'] = df_text['text'].apply(lambda x: len(x))
df_text = df_text.sort_values('text_len',ascending=False).reset_index(drop=True)
del df_text['text_len']

print('df_text.shape',df_text.shape)
print(df_text) # sort txt by its length

df_text.shape (5, 2)
             id                                               text
0  18409261F5C2  80% of Americans believe seeking multiple opin...
1  0FB0700DAF44  During a group project, have you ever asked a ...
2  DF920E0A7337  Have you ever asked more than one person for h...
3  D72CB1C11673  Making choices in life can be very difficult. ...
4  D46BCB48440A  When people ask for advice,they sometimes talk...


In [10]:
df_text

,id,text
0,18409261F5C2,80% of Americans believe seeking multiple opin...
1,0FB0700DAF44,"During a group project, have you ever asked a ..."
2,DF920E0A7337,Have you ever asked more than one person for h...
3,D72CB1C11673,Making choices in life can be very difficult. ...
4,D46BCB48440A,"When people ask for advice,they sometimes talk..."


In [11]:
def text_to_word(text):
    word = text.split()
    word_offset = []

    start = 0
    for w in word:
        r = text[start:].find(w)

        if r==-1:
            raise NotImplementedError
        else:
            start = start+r
            end   = start+len(w)
            word_offset.append((start,end))
            #print('%32s'%w, '%5d'%start, '%5d'%r, text[start:end])
        start = end

    return word, word_offset

In [12]:
def word_probability_to_predict_df(text_to_word_probability, id):
    len_word = len(text_to_word_probability)
    word_predict = text_to_word_probability.argmax(-1)
    word_score   = text_to_word_probability.max(-1)
    predict_df = []

    t = 0
    while 1:
        if word_predict[t] not in [
            discourse_marker_to_label['O'],
            discourse_marker_to_label['PAD'],
        ]:
            start = t
            b_marker_label = word_predict[t]
        else:
            t = t+1
            if t== len_word-1: break
            continue

        t = t+1
        if t== len_word-1: break

        #----
        if   label_to_discourse_marker[b_marker_label][0]=='B':
            i_marker_label = b_marker_label+1
        elif label_to_discourse_marker[b_marker_label][0]=='I':
            i_marker_label = b_marker_label
        else:
            raise NotImplementedError

        while 1:
            #print(t)
            if (word_predict[t] != i_marker_label) or (t ==len_word-1):
                end = t
                prediction_string = ' '.join([str(i) for i in range(start,end)]) #np.arange(start,end).tolist()
                discourse_type = label_to_discourse_marker[b_marker_label][2:]
                discourse_score = word_score[start:end].tolist()
                predict_df.append((id, discourse_type, prediction_string, str(discourse_score)))
                #print(predict_df[-1])
                break
            else:
                t = t+1
                continue
        if t== len_word-1: break

    predict_df = pd.DataFrame(predict_df, columns=['id', 'class', 'predictionstring', 'score'])
    return predict_df

In [13]:
def do_threshold(submit_df, use=['length','probability']):
    df = submit_df.copy()
    df = df.fillna('')

    if 'length' in use:
        df['l'] = df.predictionstring.apply(lambda x: len(x.split()))
        for key, value in length_threshold.items():
            #value=3
            index = df.loc[df['class'] == key].query('l<%d'%value).index
            df.drop(index, inplace=True)

    if 'probability' in use:
        df['s'] = df.score.apply(lambda x: np.mean(eval(x)))
        for key, value in probability_threshold.items():
            index = df.loc[df['class'] == key].query('s<%f'%value).index
            df.drop(index, inplace=True)

    df = df[['id', 'class', 'predictionstring']]
    return df

In [14]:
class FeedbackModel(tez.Model):
    def __init__(self, model_name, num_labels):
        super().__init__()
        self.model_name = model_name
        self.num_labels = num_labels
        config = AutoConfig.from_pretrained(model_name)

        hidden_dropout_prob: float = 0.1
        layer_norm_eps: float = 1e-7
        config.update(
            {
                "output_hidden_states": True,
                "hidden_dropout_prob": hidden_dropout_prob,
                "layer_norm_eps": layer_norm_eps,
                "add_pooling_layer": False,
            }
        )
        self.transformer = AutoModel.from_config(config)
        self.output = nn.Linear(config.hidden_size, self.num_labels)

    def forward(self, ids, mask):
        transformer_out = self.transformer(ids, mask)
        sequence_output = transformer_out.last_hidden_state
        logits = self.output(sequence_output)
        logits = torch.softmax(logits, dim=-1)
        return logits

In [15]:
def jn(pst, start, end):
    return " ".join([str(x) for x in pst[start:end]])

def link_evidence(oof):
    thresh = 1
    idu = oof['id'].unique()
    idc = idu[1]
    eoof = oof[oof['class'] == "Evidence"]
    neoof = oof[oof['class'] != "Evidence"]
    for thresh2 in range(26, 27, 1):
        retval = []
        for idv in idu:
            for c in ['Lead', 'Position', 'Evidence', 'Claim', 'Concluding Statement',
                      'Counterclaim', 'Rebuttal']:
                q = eoof[(eoof['id'] == idv) & (eoof['class'] == c)]
                if len(q) == 0:
                    continue
                pst = []
                for i, r in q.iterrows():
                    pst = pst + [-1] + [int(x) for x in r['predictionstring'].split()] # -1作为分割合并的间隔
                start = 1
                end = 1
                for i in range(2, len(pst)):
                    cur = pst[i]
                    end = i
                    # if pst[start] == 205:
                    #   print(cur, pst[start], cur - pst[start])
                    # Evidence的情况下：会一直延续到cur==-1的情况，然后判断下一个token是不是和之前的token仍旧满足26的最大距离要求，
                    # 如果满足就相连，否则就append到retval里
                    if (cur == -1 and c != 'Evidence') or ((cur == -1) and (
                            (pst[i + 1] > pst[end - 1] + thresh) or (pst[i + 1] - pst[start] > thresh2))): #
                        retval.append((idv, c, jn(pst, start, end)))
                        start = i + 1
                v = (idv, c, jn(pst, start, end + 1))
                # print(v)
                retval.append(v)
        roof = pd.DataFrame(retval, columns=['id', 'class', 'predictionstring'])
        roof = roof.merge(neoof, how='outer')
        return roof

In [16]:
def run_submit():
    results = []
    all_model_idx = 0

    for model_type, model_grp in model_grps.items():

        print(f'{model_type} loaded ok.\n')

        arch = model_grp['arch']
        num_models = len(model_grp['checkpoints'])

        tokenizer = AutoTokenizer.from_pretrained(arch)

        split_texts = np.array_split(df_text[['id', 'text']].values, 2)

        tokenized_text_ = Parallel(n_jobs=2, backend="multiprocessing")(
            delayed(tokenize_data)(text, tokenizer, model_grp['max_length']) for text in split_texts
        )

        tokenized_text = []

        for t in tokenized_text_:
            tokenized_text.extend(t)

        del tokenized_text_

        collate_func = Collate(tokenizer)
        valid_dataset = FeedbackDataset(tokenized_text)
        valid_loader = DataLoader(
            valid_dataset,
            sampler=SequentialSampler(valid_dataset),
            batch_size=model_grp['batch_size'],  # 4, #
            drop_last=False,
            num_workers=2,  # 0, #
            pin_memory=False,
            collate_fn = collate_func,
        )

        # start here !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!1
        results_n = {
            'id': [],
            'token_mask': [],
            'token_offset': [],
            'probability': [],
        }
       
        for model_idx, (checkpoint, name) in enumerate(model_grp['checkpoints']):
            net_template = FeedbackModel(arch, 15)
            net_template.load_state_dict(torch.load(checkpoint))
            net_template.cuda()

            print('load ok : [%d] %s' % (all_model_idx, name))
            all_model_idx += 1

            T = 0
            start_timer = timer()
            for t, batch in enumerate(valid_loader):
                batch_size = len(batch['index'])
                token_mask = batch['token_mask']
                token_id = batch['token_id']
#                 print("data_shape", token_id.shape)
                token_mask = token_mask.cuda()
                token_id = token_id.cuda()

                net_template.eval()
                with torch.no_grad():
                    with amp.autocast(enabled=is_amp):
                        probability = data_parallel(net_template, (token_id, token_mask))
                        # probability = net[n](token_id, token_mask)

                        if model_idx == 0:
                            results_n['probability'].append(
                                ((probability / num_models) * 255).byte().data.cpu().numpy())
                            results_n['token_offset'] += [eval(x) for x in batch['token_offset']]
                            results_n['token_mask'].append(token_mask.bool().data.cpu().numpy())
                            results_n['id'] += batch['id']

                        else:
                            results_n['probability'][t] = results_n['probability'][t] + (
                                        (probability / num_models) * 255).byte().data.cpu().numpy()

                        T += batch_size

                print('\r\t%d/%d  %s' % (T, len(valid_dataset), time_to_str(timer() - start_timer, 'sec')), end='',
                      flush=True)
#                 del token_id,token_mask
            del net_template,token_id,token_mask
            
            gc.collect()

            torch.cuda.empty_cache()
            print('')
       
        # ----------------------------
        del valid_dataset, valid_loader, tokenized_text
        gc.collect()
        
        print('')

        prob_list = []
        for b_idx in range(len(results_n["probability"])):
            prob_list.extend(np.split(results_n["probability"][b_idx], len(results_n["probability"][b_idx])))

        results.append({
            'probability': prob_list,
            'token_offset': np.array(results_n['token_offset'], object)
            # 'token_mask'] = np.concatenate(results['token_mask'])
            # 'id'] = np.array(results['id' ])
        })
    # ------------------------------------------------------------------------
    # results: [num_net, [prob list]]
    num_net = len(model_grps.keys())

    submit_df = []
    for i in range(num_valid):
        d = df_text.iloc[i]
        id = d.id
        text = d.text
        word, word_offset = text_to_word(text)
        # print(i,id[i], len(text), len(word))

        # ensemble -----
        token_to_text_probability = np.full((len(text), num_discourse_marker), 0, np.float32)
        for j in range(num_net):
            p = results[j]['probability'][i][0][1:] / 255 # due to np.split adding another dim [0]
            for t, (start, end) in enumerate(results[j]['token_offset'][i]):
                if t == max_length - 1: break  # assume max_length, else use token_mask to get length
                token_to_text_probability[start:end] += (p[t])  # **0.5
        token_to_text_probability = token_to_text_probability / num_net
        # ensemble -----

        text_to_word_probability = np.full((len(word), num_discourse_marker), 0, np.float32)
        for t, (start, end) in enumerate(word_offset):
            text_to_word_probability[t] = token_to_text_probability[start:end].mean(0)

        predict_df = word_probability_to_predict_df(text_to_word_probability, id)
        submit_df.append(predict_df)
        # print('\r preparing submit_df :', i, id, len(text), len(word), end ='', flush=True)
    print('')

    # ----------------------------------------
    submit_df = pd.concat(submit_df).reset_index(drop=True)
    submit_df = do_threshold(submit_df, use=['length', 'probability'])
    submit_df = link_evidence(submit_df)
    submit_df.to_csv('submission.csv', index=False)

    print('----')
    for t in range(3): print(submit_df.iloc[t], '\n')
    print('submission ok!----')

In [17]:
import warnings
warnings.filterwarnings('ignore')

In [18]:
run_submit()

deberta loaded ok.

load ok : [0] deberta_fold_0
	5/5   0 min 02 sec


----
id                                                       18409261F5C2
class                                                        Evidence
predictionstring    162 163 164 165 166 167 168 169 170 171 172 17...
Name: 0, dtype: object 

id                                                       18409261F5C2
class                                                        Evidence
predictionstring    441 442 443 444 445 446 447 448 449 450 451 45...
Name: 1, dtype: object 

id                                                       18409261F5C2
class                                                        Evidence
predictionstring    739 740 741 742 743 744 745 746 747 748 749 75...
Name: 2, dtype: object 

submission ok!----


The code below was taken from kaggle user tianmin. <a href="https://www.kaggle.com/code/tianmin/visualization-of-essay-dataset">This</a> is the link to his notebook.


So let's use some colors to make each part more outstanding.

Let's give each discourse type a color, as shown below:


<h3 style="color:rgb(255, 102, 204);">Lead</h3>
<h3 style="color:rgb(0, 0, 102);">Position</h3>
<h3 style="color:rgb(51, 102, 255);">Claim</h3>
<h3 style="color:rgb(153, 102, 51);">Counterclaim</h3>
<h3 style="color:rgb(102, 204, 255);">Rebuttal</h3>
<h3 style="color:rgb(0, 0, 0);">Evidence</h3>
<h3 style="color:rgb(51, 51, 153);">Concluding Statement</h3>

In [25]:
# iterate each discourse type and print it in the color as we wished
def color_article(id):
    train = pd.read_csv('../input/feedback-prize-2021/train.csv')
    sample = train.loc[train['id'] == id]
    for i in range(sample.shape[0]):
        text = sample['discourse_text'].iloc[i]
        discourse = sample['discourse_type'].iloc[i]
        if discourse == 'Lead':
            color = '(255, 102, 204)'
        elif discourse == 'Position':
            color = '(0, 0, 102)'
        elif discourse == 'Claim':
            color = '(51, 102, 255)'
        elif discourse == 'Counterclaim':
            color = '(153, 102, 51)'
        elif discourse == 'Rebuttal':
            color = '(102, 204, 255)'
        elif discourse == 'Evidence':
            color = '(0, 0, 0)'
        elif discourse == 'Concluding Statement':
            color = '(51, 51, 153)'

        sample_html_text = '<p style="color:rgb' + str(color) + ';">' + str(text) +'  (' +str(discourse)+') ' + '</p>'
        display(HTML(sample_html_text))
color_article("007ACE74B050")
